In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re  # Importing regex module
import random
import time
from io import StringIO

Data Collection Part 1

In [ ]:
gamesurl = "https://www.mobygames.com/platform/switch/"
r = requests.get(gamesurl)
print(r.status_code)
gamesbs = BeautifulSoup(r.text)

In [ ]:
import pandas as pd
import requests
import time


titles = []
links = []
years = []
scores = []
genres = []

base_url = gamesurl 
max_pages = 10

for page_count in range(0, max_pages + 1):
    # Construct URL for each page
    if page_count == 0:
        page_url = base_url  # First page has no /page:x/
    else:
        page_url = f"{base_url}page:{page_count}/"

    print(f"Requesting URL: {page_url}")

    response = requests.get(page_url)
    tables = pd.read_html(StringIO(response.text))  # Get all tables on the page

    # Assuming the table you want is the first one (tables[0])
    games_df = tables[0]  

    # Extract relevant columns from games_df as needed
    titles.extend(games_df['Title'].tolist())  # replace 'Title' with the actual column name
    years.extend(games_df['Released'].tolist())  # replace 'Year' with the actual column name
    scores.extend(games_df['Moby Score'].tolist())  # replace 'Score' with the actual column name
    genres.extend(games_df['Genres'].tolist())

    # If links are in another structure on the page, BeautifulSoup might still be useful:
    gamesbs = BeautifulSoup(response.text, 'html.parser')
    game_links = [tag['href'] for tag in gamesbs.select('td.text-nowrap a')]
    links.extend(game_links)

    page_count += 1
    time.sleep(3)  # Respectful delay for scraping

print(f"Collected {len(titles)} titles.")


In [ ]:
basic_games = {
    'Title': titles,
    'Score': scores,
    'Year': years,
    'Genre': genres
}

basic_games_df = pd.DataFrame(basic_games)

Data Collection Part 2

In [ ]:
game_urls = links[:250]

# List to store all extracted game data
games_data_1 = []

# Define the fields you want to extract (e.g., Genre, Perspective, Gameplay, etc.)
fields = ["Perspective", "Gameplay", "Interface", "Setting"]

# Loop through each game URL
for url in game_urls:
    # Request the HTML content of the page
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Initialize a dictionary to store the extracted data for this game
        game_info = {}

        ## Extract Release Date (no cleaning applied here)
        for dt in soup.select('.info-release dt'):
            if dt.text.strip() == "Released":
                game_info['Released'] = dt.find_next_sibling('dd').text.strip()

        ## Extract Critics (no cleaning applied here)
        for dt in soup.select('.info-score dt'):
            if dt.text.strip() == "Critics":
                game_info['Critics'] = dt.find_next_sibling('dd').text.strip()

        ## Extract other fields and save as raw data (no cleaning)
        for field in fields:
            for dt in soup.select('.info-genres dt'):
                if dt.text.strip() == field:
                    game_info[field] = dt.find_next_sibling('dd').text.strip()

        ## Extract ESRB Rating (no cleaning applied)
        for dt in soup.select('.info-specs dt'):
            if dt.text.strip() == "ESRB Rating":
                game_info['ESRB Rating'] = dt.find_next_sibling('dd').text.strip()

        ## Extract Multiplayer Options (no cleaning applied)
        for dt in soup.select('.info-specs dt'):
            if dt.text.strip() == "Multiplayer Options":
                game_info['Multiplayer Options'] = dt.find_next_sibling('dd').text.strip()

        # Add the game data to the list
        games_data_1.append(game_info)

    else:
        print(f"Failed to retrieve {url}")

    # Sleep for a random time between 3 and 5 seconds to avoid being blocked
    time.sleep(random.randint(3, 5))

# Convert the list of dictionaries into a DataFrame
games_df_1 = pd.DataFrame(games_data_1)

# Print the DataFrame (you can clean the data afterwards)
print(games_df_1)



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

game_urls = links[250:]

# List to store all extracted game data
games_data_2 = []

# Define the fields you want to extract (e.g., Genre, Perspective, Gameplay, etc.)
fields = ["Perspective", "Gameplay", "Interface", "Setting"]

# Loop through each game URL
for url in game_urls:
    # Request the HTML content of the page
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Initialize a dictionary to store the extracted data for this game
        game_info = {}

        ## Extract Release Date (no cleaning applied here)
        for dt in soup.select('.info-release dt'):
            if dt.text.strip() == "Released":
                game_info['Released'] = dt.find_next_sibling('dd').text.strip()

        ## Extract Critics (no cleaning applied here)
        for dt in soup.select('.info-score dt'):
            if dt.text.strip() == "Critics":
                game_info['Critics'] = dt.find_next_sibling('dd').text.strip()

        ## Extract other fields and save as raw data (no cleaning)
        for field in fields:
            for dt in soup.select('.info-genres dt'):
                if dt.text.strip() == field:
                    game_info[field] = dt.find_next_sibling('dd').text.strip()

        ## Extract ESRB Rating (no cleaning applied)
        for dt in soup.select('.info-specs dt'):
            if dt.text.strip() == "ESRB Rating":
                game_info['ESRB Rating'] = dt.find_next_sibling('dd').text.strip()

        ## Extract Multiplayer Options (no cleaning applied)
        for dt in soup.select('.info-specs dt'):
            if dt.text.strip() == "Multiplayer Options":
                game_info['Multiplayer Options'] = dt.find_next_sibling('dd').text.strip()

        # Add the game data to the list
        games_data_2.append(game_info)

    else:
        print(f"Failed to retrieve {url}")

    # Sleep for a random time between 3 and 5 seconds to avoid being blocked
    time.sleep(random.randint(3, 5))

# Convert the list of dictionaries into a DataFrame
games_df_2 = pd.DataFrame(games_data_2)

# Print the DataFrame (you can clean the data afterwards)
print(games_df_2)



In [ ]:
combined_games_df = pd.concat([games_df_1, games_df_2], ignore_index=True)
combined_games_df

Data Cleaning

In [365]:
# Function to clean unwanted characters like newline, commas, and slashes
def clean_value(value):
    return re.sub(r'[\n,\/]+', ' ', value).strip()

def clean_and_split(value):
    
    # Remove unwanted characters like newline, commas, and slashes
    cleaned_value = re.sub(r'[\n]+', ' ', value).strip()

    # Remove any text in parentheses
    cleaned_value = re.sub(r'\s*\(.*?\)', '', cleaned_value)

    cleaned_value = re.sub(r'RPG', 'Role-playing', cleaned_value)

    # Replace slashes with ampersands
    # cleaned_value = cleaned_value.replace('/', '&')

    # Add spaces between patterns like "1st-person3rd-person" so they can be split
    cleaned_value = re.sub(r'(\d\w*-person)(?=\d)', r'\1 ', cleaned_value)

    # Split based on capital letters, spaces, or common delimiters (like slashes)
    split_values = re.split(r'(?=[A-Z])|\s*[,]+\s*|\s(?=[0-9])', cleaned_value)
    
    # Remove extra spaces
    return [val.strip() for val in split_values if val.strip()]

def clean_genres(value):
    # Check if value is a string
    if isinstance(value, str):

        # Split based on commas and remove extra spaces
        cleaned_value = [genre.strip() for genre in cleaned_value.split(',')]
        
        return cleaned_value
    return value  

def clean_critics(critics):
    critics = re.sub(r'\s?\(\d+\)', '', critics)  # Remove numbers in parentheses
    critics = re.sub(r'%', '', critics)  # Remove percentage sign
    critics = critics.strip()  # Remove any remaining unwanted whitespace
    return critics

# Only get month
def clean_released_date(value):
    return value.split()[0]


In [ ]:
# Function to split values based on capital letters or delimiters
games_df = combined_games_df

games_df_clean = games_df.copy()

# Clean specific columns using .apply()
games_df_clean['Released'] = games_df['Released'].apply(clean_released_date)
games_df_clean['Critics'] = games_df['Critics'].apply(clean_value).apply(clean_critics)
games_df_clean['Gameplay'] = games_df['Gameplay'].apply(lambda x: clean_and_split(x) if isinstance(x, str) else x)
games_df_clean['Setting'] = games_df['Setting'].apply(lambda x: clean_and_split(x) if isinstance(x, str) else x)
games_df_clean['Perspective'] = games_df['Perspective'].apply(lambda x: clean_and_split(x) if isinstance(x, str) else x)
games_df_clean['Interface'] = games_df['Interface'].apply(lambda x: clean_and_split(x) if isinstance(x, str) else x)

# Split the 'Genre' column based on commas and remove extra spaces
basic_games_df['Genre'] = basic_games_df['Genre'].apply(clean_genres)

EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sns.set_theme(style="ticks", palette="colorblind")


In [ ]:
all_combined_df = pd.concat([basic_games_df, games_df_clean], axis = 1)

# Convert 'Year' column to categorical type
all_combined_df['Year'] = all_combined_df['Year'].astype('category')

# Convert the column 'Score' to numeric
all_combined_df['Critics'] = pd.to_numeric(all_combined_df['Critics'], errors='coerce')


# Count missing values for each column
missing_values_per_column = all_combined_df.isna().sum()
missing_values_per_column

#ESRB+ and Multiplayer Options have too many missing values to be of worth; drop them
all_combined_df = all_combined_df.drop(['ESRB Rating', 'Multiplayer Options'], axis=1)

In [ ]:
# Save to  csv
all_combined_df.to_csv('games_data.csv', index = False)

In [ ]:
## Barplot

# List of month names
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
          'July', 'August', 'September', 'October', 'November', 'December']

# Filter rows where 'Released' contains a month name
barplot_data = all_combined_df[all_combined_df['Released'].str.contains('|'.join(month_order), na=False)]

# Convert the 'Released' column to a categorical type with the defined order
barplot_data['Released'] = pd.Categorical(barplot_data['Released'], categories=month_order, ordered=True)

# Plotting the count plot with months in chronological order
plt.figure(figsize=(12, 6))
sns.countplot(data=barplot_data, x='Released', order=month_order)
plt.title('Releases per Month')
plt.xlabel('Release Month')
plt.ylabel('Count')
plt.xticks(rotation=45)
#plt.tight_layout()
plt.show()

In [ ]:
## Boxplot

# Explode the 'Genre' column to match each perspective with its corresponding 'Critics' score
exploded_df = all_combined_df.explode('Genre').reset_index(drop = True)
# .explode('Gameplay').explode('Interface').explode('Setting').explode('Genre')

# Plot a box plot of Critics scores for each Perspective
sns.set_theme(style="ticks", palette="colorblind")
sns.boxplot(x='Genre', y='Score', data=exploded_df)
plt.xticks(rotation=90)
# plt.xlabel('Perspective')
plt.ylabel('Moby Score')
plt.title('Score by Genre Category')
plt.show()

In [ ]:
## Stacked DataFrame

# Group the data and create a stacked DataFrame
exploded_df = all_combined_df.explode('Genre').reset_index(drop = True)
data = exploded_df.groupby(['Year', 'Genre']).size().reset_index(name='Count')

# Create the stacked area chart
fig = px.area(data, x='Year', y='Count', color='Genre', 
              title='Games Released by Genre over Time',
              labels={'Year': 'Year', 'Count': 'Number of Games'},
              template='plotly')

fig.update_layout(margin=dict(l=0, r=0, t=40, b=40))
fig.show()


In [ ]:
## Stacked bar plot

exploded_game_inter = all_combined_df.explode('Setting').explode('Perspective').reset_index(drop = True)

# Pivot the data to create a stacked bar chart
pivot_df = exploded_game_inter.groupby(['Setting', 'Perspective']).size().unstack(fill_value=0)

# Plot stacked bar chart
pivot_df.plot(kind='bar', stacked=True, figsize=(12, 6))

plt.title('Stacked Bar Plot: Setting and Perspective Comparison')
plt.xlabel('Settings')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

Summary Stats

In [420]:
# Summary statistics for all numeric columns

numeric_summary = all_combined_df.describe()

print(numeric_summary)


            Score     Critics
count  550.000000  550.000000
mean     8.023273   84.336364
std      0.257217    3.365087
min      7.800000   77.000000
25%      7.900000   82.000000
50%      7.900000   84.000000
75%      8.100000   86.000000
max      9.400000   96.000000
